In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from random import gauss, randrange, random

In [ ]:
%matplotlib inline

In [ ]:
import statsmodels.graphics.tsaplots as smt


### Dados Governamentais 

In [ ]:
dados_analfabetismo = pd.read_excel('analfabetismo.xlsx', index_col='Ano', parse_dates=True)
dados_analfabetismo.head()

In [ ]:
dados_analfabetismo.plot(figsize=(15,6), lw=4, ls=':', c='red')
#figsize = dimensão da figura
#lw = largura da linha 
#ls = formato dos pontos

In [ ]:
dados_analfabetismo.plot.hist(bins=20, edgecolor='k')

### Dados de Produção

In [ ]:
## Produção de óleo no Brasil
dados_oleo = pd.read_excel('oil-production-tonnes.xls', index_col='Year')
dados_oleo.head()

In [ ]:
dados_oleo.plot(figsize=(15,6), lw=4, ls=':', c='green')

### Dados Naturais

In [ ]:
## Série Sunspot
sunspot = pd.read_csv('sunspot.txt', header=None)
sunspot.columns = ['Sunspot']

In [ ]:
sunspot.head()

In [ ]:
sunspot.plot(figsize=(15,6), lw=4, ls='-', c='green')

### Dados Financeiros

In [ ]:
microsoft = pd.read_csv('MSFT.csv', delimiter=';', index_col='Date')
microsoft.head()

In [ ]:
microsoft['Adj_Close'].plot(figsize=(15,6), lw=2, ls='-', c='red')

### Dados Medicina 

In [ ]:
#serie de frequência cardiaca
fc_serie = pd.read_csv('hr.txt', header=None)
fc_serie.columns = ['Freq']

In [ ]:
fc_serie.plot(figsize=(15,6), lw=2, ls='-', c='black')

## Modelos Simples 

### Ruído I.I.D 

* Modelo mais simples;
* Sem componentes de tendência e sazonalidade;
* As observações são resultados de variáveis aleátorias i.i.d com média zero;
* Não existe dependência entre as observações;
* Apesar de não ser interessante para predição, é um modelo que ajuda na construção e entendimento de modelos mais complexos;

In [ ]:
## Série gerada por Ruído I.I.D Gaussiano 
dados_iid = [gauss(0.0, 1.0) for i in range(500)]
serie_iid = pd.Series(dados_iid)

In [ ]:
serie_iid.plot(figsize=(15,6), lw=2, ls='-', c='black')

### Processo Binário
* As observações só podem assumir dois valores possivéis

In [ ]:
serie_vitorias = pd.read_csv('vitorias.txt', header=None)

In [ ]:
serie_vitorias.columns = ['Vitoria']
serie_vitorias.plot(figsize=(15,6), lw=2, ls='-', c='black')
plt.show()

#### Random Walk

* Idéia remete a “caminhada do bêbado”. Tomada de vários passos consecutivos, cada qual em uma direção aleatória;
* O valor atual é obtido através de valor/valores passados + um ruído: $S_{t} = S_{t-1} + X_{t}$
* $X_{t}$ é um ruído i.i.d

In [ ]:
serie_randomwalk = [np.random.normal(0)]
tam_serie = 500
for i in range(1, tam_serie):
    valor = serie_randomwalk[i-1] + np.random.normal(0)
    serie_randomwalk.append(valor)

In [ ]:
serie_randomwalk = pd.Series(serie_randomwalk)
serie_randomwalk.plot(figsize=(15,6), lw=2, ls='-', c='black')

## Modelos com tendência e sazonalidade
* Algumas séries é perceptível que não podem ser modeladas por modelos simples de média zero;
* São geradas com componentes de tendência e sazonalidade;
* __Tendência__: mudança sistemática na série temporal que não parece ser periódico;
* __Sazonalidade__: comportamento que se repete durante um período; 

$X_{t} = m_{t} + s_{t} + Y_{t}$


###  Modelo com Tendência

$$X_{t} = m_{t} + Y_{t}$$

$m_{t}$ é um componente de tendência e pode ser estimado através do método de mínimo quadrado:

$$m_{t} = a_{0}+a_{1}t+ a_{2}t^2$$

Os valores de $a$ são ajustados para minimizar a função: 
$$\sum_{t=1}^{n} (x_{t} - m_{t})^2$$


### Modelo com Sazonalidade 

$$X_{t} = s_{t} + Y_{t}$$

$S_{t}$ é o componente de sazonalidade, resultado de uma função periódica que pode ser obtida através da soma de ondas senoidais:

$$s_{t} = \alpha_{0} + \sum_{j=1}^{k} (a_{j} cos(\lambda_{j}t) + b_{j}sin(\lambda_{j}t))$$ 

$a$ e $b$ são parâmetros desconhecidos e $\lambda$ são as frequências

## Uma abordagem Geral para modelagem de séries temporais

### >Plotar a série e analisar alguns aspectos:
* Tendência
* Componentes sazonais
* Alterações acentuadas no comportamento
* Observações discrepantes com os dados

### >Remover tendências e componentes sazonais para obter resíduos estacionários;

### >Escolher um modelo para ajustar aos resíduos;

### >Alcançar a previsão original da série através da previsão dos componentes;

## -------------------------------

In [ ]:
serie_airline = pd.read_csv('airline.txt', header=None)
serie_airline.columns = ['Airline']
#é preciso criar uma frequencia na série para usar a função de decomposição 
rng = pd.date_range('1949 Jan 1', periods = len(serie_airline), freq = 'M')
serie_airline = pd.Series(serie_airline['Airline'].values, index = rng)

In [ ]:
serie_airline.plot(figsize=(15,6))

In [ ]:
m_m = serie_airline.rolling(window=12).mean()
m_std = serie_airline.rolling(window=12).std() 

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6


orig = plt.plot(serie_airline, color='blue',label='Original')
mean = plt.plot(m_m, color='red',label='Média')
std = plt.plot(m_std, color='black',label='Desvio')
plt.legend(loc='best')


In [ ]:
serie_airline.plot.hist(edgecolor='k')

In [ ]:
import statsmodels.api as sm

decomposition = sm.tsa.seasonal_decompose(serie_airline, model='additive')
fig = decomposition.plot()
plt.show()

In [ ]:
plt.plot(decomposition.seasonal)

In [ ]:
plt.plot(decomposition.trend)

In [ ]:
plt.plot(decomposition.resid)

### Estimando e Eliminando a tendência 

In [ ]:
tendencia = decomposition.trend
serie_sem_tendencia = serie_airline - tendencia

In [ ]:
m_m = serie_sem_tendencia.rolling(window=12).mean()
mean = plt.plot(m_m, color='red',label='Média')
plt.plot(serie_sem_tendencia, label='Sem tendencia')
plt.legend(loc='best')
plt.show()

In [ ]:
serie_sem_tendencia.plot.hist(edgecolor='k')

### Estimando e eliminando sazonalidade

In [ ]:
sazonalidade = decomposition.seasonal
ruido = serie_sem_tendencia - sazonalidade

In [ ]:
m_m = ruido.rolling(window=12).mean()
mean = plt.plot(m_m, color='red',label='Média')
plt.plot(ruido, label='ruído')
plt.legend(loc='best')
plt.show()

#### Multiplicativo

In [ ]:
decomposition = sm.tsa.seasonal_decompose(serie_airline, model='multiplicative')
fig = decomposition.plot()
plt.show()

In [ ]:
ruido_multi = decomposition.resid
ruido_multi.plot.hist(edgecolor='k')

### ______________________

## Processo estacionario 

In [ ]:
star = pd.read_csv('star.txt', header=None)

In [ ]:
star.plot()

In [ ]:
star.plot.hist(edgecolor='k')

In [ ]:
serie_airline.plot()

In [ ]:
serie_airline.plot.hist(edgecolor='k')

In [ ]:
def acf_pacf(x, qtd_lag):
    fig = plt.figure(figsize=(16,10))
    ax1 = fig.add_subplot(221)
    #plot acf
    fig = smt.plot_acf(x, lags=qtd_lag, ax=ax1)
    ax2 = fig.add_subplot(222)
    #plot pacf
    fig = smt.plot_pacf(x, lags=qtd_lag, ax=ax2)
    plt.show()

In [ ]:
acf_pacf(star, 20)

In [ ]:
acf_pacf(serie_airline, 20)

### Testes estatisticos

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

In [ ]:
def teste_df(serie):
    #H0: série não estacionária 
    dftest = adfuller(serie, autolag='AIC')
    df_output = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags','Nº Observações'])
    for key,value in dftest[4].items():
        df_output['Valor Crítico (%s)'%key] = value
    print(df_output)
    if df_output[0] < df_output['Valor Crítico (5%)']: # descarta a H0 se o teste estatistico for menor que o valor crítico 
        
        print('estacionária')
    else:
        print(df_output[0])
        print('não estacionária')

In [ ]:
def teste_kpss(serie):
    #H0: série é estacionária 
    kptest = kpss(serie, regression='c')
    kp_output = pd.Series(kptest[0:3], index=['Test Statistic','p-value','#Lags'])
    for key,value in kptest[3].items():
        kp_output['Valor Crítico (%s)'%key] = value
    print(kp_output)
    if kp_output[0] > kp_output['Valor Crítico (5%)']: # descarta a H0 se o teste estatistico for MAIOR que o valor crítico 
        
        print('não estacionária')
    else:
        print(kp_output[0])
        print('estacionária')

In [ ]:
teste_df(star[0].values)

In [ ]:
teste_df(serie_airline.values)

In [ ]:
teste_kpss(star[0].values)

In [ ]:
teste_kpss(serie_airline.values)